In [2]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from IPython.display import Image, display
from pandas.tools.plotting import scatter_matrix 
%matplotlib inline


import category_encoders as ce

In [5]:
chunksize = 5000
counter = 0
csv_path = "D:\\tempd\\train_v2.csv"

JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

chunt_itr = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     chunksize=chunksize)

for chunk in chunt_itr:
    process(chunk)
print('done chunking')

NameError: name 'process' is not defined

In [7]:
def process(chunk):
    global raw_data
    filtered_chunk = chunk[(chunk.date > 20160000) & (chunk.date < 20170000)]
    raw_data = raw_data.append(filtered_chunk, ignore_index=True)

In [8]:
for column in JSON_COLUMNS:
    column_as_df = json_normalize(raw_data[column])
    column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
    raw_data = raw_data.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

KeyError: 'device'

In [ ]:
raw_data.to_csv("train16.csv", index=False)

In [3]:
raw_data = pd.read_csv('train16.csv')

C:\Users\Hawk\Miniconda3\envs\ml\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype('float')
gdf = train_df.groupby("fullVisitorId")["totals.transactionRevenue"].sum().reset_index()

#plt.figure(figsize=(8,6))
#plt.scatter(range(gdf.shape[0]), np.sort(np.log1p(gdf["totals.transactionRevenue"].values)))
#plt.xlabel('index', fontsize=12)
#plt.ylabel('TransactionRevenue', fontsize=12)
#plt.show()

In [6]:
nzi = pd.notnull(train_df["totals.transactionRevenue"]).sum()
nzr = (gdf["totals.transactionRevenue"]>0).sum()
print("Number of instances in train set with non-zero revenue : ", nzi, " and ratio is : ", nzi / train_df.shape[0])
print("Number of unique customers with non-zero revenue : ", nzr, "and the ratio is : ", nzr / gdf.shape[0])

Number of instances in train set with non-zero revenue :  5164  and ratio is :  0.01183337037945155
Number of unique customers with non-zero revenue :  4573 and the ratio is :  0.012888588258504552


In [7]:
const_cols = [c for c in train_df.columns if train_df[c].nunique(dropna=False)==1 ]
const_cols

['socialEngagementType',
 'device.browserSize',
 'device.browserVersion',
 'device.flashVersion',
 'device.language',
 'device.mobileDeviceBranding',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.operatingSystemVersion',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'totals.visits',
 'trafficSource.adwordsClickInfo.criteriaParameters']

In [24]:
train_df.head()
print(train_df.shape)

(436393, 39)


In [10]:
cols_to_drop = const_cols
train_df = train_df.drop(cols_to_drop + ["trafficSource.campaignCode"], axis=1)

In [12]:
train_df["totals.transactionRevenue"].fillna(0, inplace=True)

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436393 entries, 0 to 436392
Data columns (total 39 columns):
channelGrouping                                 436393 non-null object
customDimensions                                436393 non-null object
date                                            436393 non-null int64
fullVisitorId                                   436393 non-null uint64
hits                                            436393 non-null object
visitId                                         436393 non-null int64
visitNumber                                     436393 non-null int64
visitStartTime                                  436393 non-null int64
device.browser                                  436393 non-null object
device.deviceCategory                           436393 non-null object
device.isMobile                                 436393 non-null bool
device.operatingSystem                          436393 non-null object
geoNetwork.city                            

In [20]:
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

In [28]:
### Feature selection and encoding
y = train_df['totals.transactionRevenue']

ce_ord = ce.OrdinalEncoder(cols = cat_cols, handle_unknown='impute')

train_df = ce_ord.fit_transform(train_df, y)

In [33]:
train_df.columns

Index(['channelGrouping', 'customDimensions', 'date', 'fullVisitorId', 'hits',
       'visitId', 'visitNumber', 'visitStartTime', 'device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.timeOnSite',
       'totals.totalTransactionRevenue', 'totals.transactionRevenue',
       'totals.transactions', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', '

In [31]:
num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    train_df[col] = train_df[col].astype(float)

In [32]:
train_df.to_csv("train-flattened-drop.csv", index=False)

In [3]:
raw_data = pd.read_csv('train-flattened-drop.csv', 
                     dtype={'fullVisitorId': 'str'})

In [5]:
train_df = raw_data

In [6]:
train_df.drop(['customDimensions'], axis=1, inplace=True)

In [7]:
train_df.head()

,channelGrouping,date,fullVisitorId,hits,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,1,20160902,9499785259412240342,"[{'hitNumber': '1', 'time': '0', 'hour': '3', ...",1472812272,1.0,1.472812e+09,1,1,False,...,1,1,0,1,1,1,1,1,1,1
1,1,20160902,3696906537737368442,"[{'hitNumber': '1', 'time': '0', 'hour': '15',...",1472856874,1.0,1.472857e+09,2,1,False,...,1,1,0,1,1,1,1,1,1,1
2,1,20160902,5688512451361930466,"[{'hitNumber': '1', 'time': '0', 'hour': '3', ...",1472811524,1.0,1.472812e+09,2,1,False,...,1,1,0,1,1,1,1,1,1,1
3,1,20160902,9716453263501147451,"[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",1472823508,1.0,1.472824e+09,3,2,True,...,1,1,0,1,1,1,1,1,1,1
4,1,20160902,5330453591145100432,"[{'hitNumber': '1', 'time': '0', 'hour': '11',...",1472839619,1.0,1.472840e+09,2,1,False,...,1,1,0,1,1,1,1,1,1,1


In [18]:
train_df.drop(['hits'], axis=1, inplace=True)

In [20]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436393 entries, 0 to 436392
Data columns (total 37 columns):
channelGrouping                                 436393 non-null int64
date                                            436393 non-null int64
fullVisitorId                                   436393 non-null object
visitId                                         436393 non-null int64
visitNumber                                     436393 non-null float64
visitStartTime                                  436393 non-null float64
device.browser                                  436393 non-null int64
device.deviceCategory                           436393 non-null int64
device.isMobile                                 436393 non-null bool
device.operatingSystem                          436393 non-null int64
geoNetwork.city                                 436393 non-null int64
geoNetwork.continent                            436393 non-null int64
geoNetwork.country                           

In [21]:
train_df.to_csv("train-flattened-drop.csv", index=False)